In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
from haversine import haversine

In [38]:
coordinate = pd.read_csv('CMcoordinate_month.csv', encoding='cp949')
coordinate = coordinate[['Unnamed: 0','Unnamed: 1','2022.06']]
coordinate.head()

,Unnamed: 0,Unnamed: 1,2022.06
0,서울특별시,위도,37.551332
1,서울특별시,경도,126.991301
2,부산광역시,위도,35.166384
3,부산광역시,경도,129.068013
4,대구광역시,위도,35.852187


In [39]:
coordinate = coordinate.pivot(index='Unnamed: 0', columns='Unnamed: 1', values='2022.06')
coordinate = coordinate.reset_index() 
coordinate.columns = ['지역', '위도', '경도']
coordinate.head()

,지역,위도,경도
0,강원도,128.253846,37.665886
1,경기도,126.998239,37.431045
2,경상남도,128.569942,35.202801
3,경상북도,128.772876,36.154651
4,광주광역시,126.873806,35.151763


In [47]:
mig = pd.read_csv('migration_month.csv', encoding='cp949')
mig.head()

,전출지별,전입지별,2017.01,2017.02,2017.03,2017.04,2017.05,2017.06,2017.07,2017.08,...,2021.09,2021.1,2021.11,2021.12,2022.01,2022.02,2022.03,2022.04,2022.05,2022.06
0,서울특별시,서울특별시,73576,103646,95471,79107,88260,80936,79579,92857,...,73846,74952,74487,74352,62740,86873,76912,63149,69274,58766
1,서울특별시,부산광역시,1234,1801,1533,1131,1060,1100,1153,1393,...,1154,819,915,1137,1085,1465,1211,848,874,865
2,서울특별시,대구광역시,814,1279,1041,664,667,592,641,763,...,733,538,601,756,716,997,867,608,586,535
3,서울특별시,인천광역시,3134,4403,3800,3123,3379,3147,3153,3576,...,4121,3776,3602,3793,3032,4531,4126,3199,3835,3341
4,서울특별시,광주광역시,800,965,813,590,631,581,598,714,...,712,462,494,592,654,743,674,436,462,415


In [49]:
data1 = pd.merge(mig, coordinate, how = 'outer', left_on='전출지별', right_on='지역')
data1 = data1.drop(['지역'], axis=1)
data1 = data1.rename(columns = {
    '위도' : '전출 위도',
    '경도' : '전출 경도'
})
data1.head()

,전출지별,전입지별,2017.01,2017.02,2017.03,2017.04,2017.05,2017.06,2017.07,2017.08,...,2021.11,2021.12,2022.01,2022.02,2022.03,2022.04,2022.05,2022.06,전출 위도,전출 경도
0,서울특별시,서울특별시,73576,103646,95471,79107,88260,80936,79579,92857,...,74487,74352,62740,86873,76912,63149,69274,58766,126.991301,37.551332
1,서울특별시,부산광역시,1234,1801,1533,1131,1060,1100,1153,1393,...,915,1137,1085,1465,1211,848,874,865,126.991301,37.551332
2,서울특별시,대구광역시,814,1279,1041,664,667,592,641,763,...,601,756,716,997,867,608,586,535,126.991301,37.551332
3,서울특별시,인천광역시,3134,4403,3800,3123,3379,3147,3153,3576,...,3602,3793,3032,4531,4126,3199,3835,3341,126.991301,37.551332
4,서울특별시,광주광역시,800,965,813,590,631,581,598,714,...,494,592,654,743,674,436,462,415,126.991301,37.551332


In [50]:
data2 = pd.merge(data1, coordinate, how = 'outer', left_on='전입지별', right_on='지역')
data2 = data2.drop(['지역'], axis=1)
data2 = data2.rename(columns = {
    '위도' : '전입 위도',
    '경도' : '전입 경도'
})
data2.head()

,전출지별,전입지별,2017.01,2017.02,2017.03,2017.04,2017.05,2017.06,2017.07,2017.08,...,2022.01,2022.02,2022.03,2022.04,2022.05,2022.06,전출 위도,전출 경도,전입 위도,전입 경도
0,서울특별시,서울특별시,73576,103646,95471,79107,88260,80936,79579,92857,...,62740,86873,76912,63149,69274,58766,126.991301,37.551332,126.991301,37.551332
1,부산광역시,서울특별시,1800,2871,2236,1413,1380,1398,1330,1779,...,1674,2886,2145,1350,1474,1393,129.068013,35.166384,126.991301,37.551332
2,대구광역시,서울특별시,1213,2008,1550,977,1009,856,844,1166,...,1206,2107,1601,983,1091,986,128.570672,35.852187,126.991301,37.551332
3,인천광역시,서울특별시,2704,3761,3207,2540,2858,2567,2506,2810,...,2620,3723,3253,2410,2832,2517,126.688008,37.491375,126.991301,37.551332
4,광주광역시,서울특별시,1008,1562,1261,786,761,757,709,950,...,805,1490,1127,728,743,712,126.873806,35.151763,126.991301,37.551332


In [54]:
data2.loc[1,'전입 위도']

126.9913011

In [58]:
dist = []
for i in range(data2.shape[0]):
    tem = haversine( (data2.loc[i,'전입 위도'], data2.loc[i,'전입 경도']),  (data2.loc[i,'전출 위도'],  data2.loc[i,'전출 경도']) )
    dist.append(tem)

In [74]:
data3 = data2.drop(['전입 위도','전입 경도','전출 경도','전출 위도', ], axis=1)
data3['거리'] = dist
data3  = data3.loc[~(data3==0.0).any(axis=1)]
data3.head()

,전출지별,전입지별,2017.01,2017.02,2017.03,2017.04,2017.05,2017.06,2017.07,2017.08,...,2021.1,2021.11,2021.12,2022.01,2022.02,2022.03,2022.04,2022.05,2022.06,거리
1,부산광역시,서울특별시,1800,2871,2236,1413,1380,1398,1330,1779,...,1261,1298,1659,1674,2886,2145,1350,1474,1393,282.837224
2,대구광역시,서울특별시,1213,2008,1550,977,1009,856,844,1166,...,910,922,1196,1206,2107,1601,983,1091,986,210.319693
3,인천광역시,서울특별시,2704,3761,3207,2540,2858,2567,2506,2810,...,2306,2526,2511,2620,3723,3253,2410,2832,2517,33.960794
4,광주광역시,서울특별시,1008,1562,1261,786,761,757,709,950,...,655,658,732,805,1490,1127,728,743,712,160.849306
5,대전광역시,서울특별시,1431,2195,1590,948,924,995,928,1206,...,804,838,1152,1252,2135,1564,892,927,985,92.846502


In [100]:
areal = pd.read_csv('areal.csv', encoding='cp949')
areal.head()

,시도(1),시군구(1),면적
0,서울,서울,605243961.2
1,서울,종로구,23912935.7
2,서울,중구,9960262.0
3,서울,용산구,21866145.2
4,서울,성동구,16859898.9


In [101]:
areal = areal.loc[ np.where(areal['시도(1)']==areal['시군구(1)'])[0], :]
areal = areal.drop('시도(1)', axis=1)
areal = areal.rename(columns = {
    '시군구(1)' : '지역'
})
areal

,지역,면적
0,서울,6.052440e+08
28,부산,7.701708e+08
48,대구,8.836982e+08
55,인천,1.066465e+09
67,광주,5.011132e+08
72,대전,5.395036e+08
79,울산,1.062328e+09
81,경기,1.019673e+10
131,강원,1.682967e+10
150,충북,7.406989e+09


In [102]:
areal.loc[80] = ['세종', 464918346.3]
areal

,지역,면적
0,서울,6.052440e+08
28,부산,7.701708e+08
48,대구,8.836982e+08
55,인천,1.066465e+09
67,광주,5.011132e+08
72,대전,5.395036e+08
79,울산,1.062328e+09
81,경기,1.019673e+10
131,강원,1.682967e+10
150,충북,7.406989e+09


In [103]:
areal['지역'] = ['서울특별시',
              '부산광역시',
              '대구광역시',
              '인천광역시',
              '광주광역시',
              '대전광역시',
              '울산광역시',
              '경기도',
              '강원도',
              '충청북도',
              '충청남도',
              '전라북도',
              '전라남도',
              '경상북도',
              '경상남도',
              '제주특별자치도',
              '세종특별자치시']
areal


,지역,면적
0,서울특별시,6.052440e+08
28,부산광역시,7.701708e+08
48,대구광역시,8.836982e+08
55,인천광역시,1.066465e+09
67,광주광역시,5.011132e+08
72,대전광역시,5.395036e+08
79,울산광역시,1.062328e+09
81,경기도,1.019673e+10
131,강원도,1.682967e+10
150,충청북도,7.406989e+09


In [104]:
data4 = pd.merge(data3, areal, how = 'outer', left_on='전입지별', right_on='지역')
data4 = data4.drop(['지역'], axis=1)
data4 = data4.rename(columns = {
    '면적' : '전입지 면적',
    '경도' : '전입 경도'
})
data4.head()

,전출지별,전입지별,2017.01,2017.02,2017.03,2017.04,2017.05,2017.06,2017.07,2017.08,...,2021.11,2021.12,2022.01,2022.02,2022.03,2022.04,2022.05,2022.06,거리,전입지 면적
0,부산광역시,서울특별시,1800,2871,2236,1413,1380,1398,1330,1779,...,1298,1659,1674,2886,2145,1350,1474,1393,282.837224,605243961.2
1,대구광역시,서울특별시,1213,2008,1550,977,1009,856,844,1166,...,922,1196,1206,2107,1601,983,1091,986,210.319693,605243961.2
2,인천광역시,서울특별시,2704,3761,3207,2540,2858,2567,2506,2810,...,2526,2511,2620,3723,3253,2410,2832,2517,33.960794,605243961.2
3,광주광역시,서울특별시,1008,1562,1261,786,761,757,709,950,...,658,732,805,1490,1127,728,743,712,160.849306,605243961.2
4,대전광역시,서울특별시,1431,2195,1590,948,924,995,928,1206,...,838,1152,1252,2135,1564,892,927,985,92.846502,605243961.2


In [106]:
# data4.to_csv('tem2.csv', encoding='utf-8-sig')

In [107]:
pop = pd.read_csv('population.csv', encoding='cp949')
pop.head()

,행정구역(시군구)별,2017.01,2017.02,2017.03,2017.04,2017.05,2017.06,2017.07,2017.08,2017.09,...,2021.09,2021.1,2021.11,2021.12,2022.01,2022.02,2022.03,2022.04,2022.05,2022.06
0,서울특별시,9930478,9931412,9926968,9922745,9919016,9914381,9908612,9902072,9891448,...,9542256,9532428,9520880,9509458,9505926,9508451,9506778,9500480,9496887,9494807
1,부산광역시,3496779,3495478,3493041,3491282,3489212,3486570,3484836,3484183,3482804,...,3358763,3356311,3353627,3350380,3348874,3347396,3343504,3340214,3338167,3336737
2,대구광역시,2484688,2484578,2483488,2483105,2482476,2481985,2481489,2479887,2478236,...,2392787,2390721,2388230,2385412,2383858,2381095,2380494,2378573,2376676,2375306
3,인천광역시,2944009,2944266,2944630,2945305,2946270,2946915,2947803,2948014,2947298,...,2941795,2945009,2946319,2948375,2949150,2950978,2952699,2953260,2955167,2957066
4,광주광역시,1469583,1468527,1467766,1467002,1466741,1466451,1466415,1466642,1466639,...,1442827,1442454,1442138,1441611,1441636,1440032,1438463,1436916,1436012,1435378


In [111]:
pop.columns

Index(['행정구역(시군구)별', '2017.01', '2017.02', '2017.03', '2017.04', '2017.05',
       '2017.06', '2017.07', '2017.08', '2017.09', '2017.1', '2017.11',
       '2017.12', '2018.01', '2018.02', '2018.03', '2018.04', '2018.05',
       '2018.06', '2018.07', '2018.08', '2018.09', '2018.1', '2018.11',
       '2018.12', '2019.01', '2019.02', '2019.03', '2019.04', '2019.05',
       '2019.06', '2019.07', '2019.08', '2019.09', '2019.1', '2019.11',
       '2019.12', '2020.01', '2020.02', '2020.03', '2020.04', '2020.05',
       '2020.06', '2020.07', '2020.08', '2020.09', '2020.1', '2020.11',
       '2020.12', '2021.01', '2021.02', '2021.03', '2021.04', '2021.05',
       '2021.06', '2021.07', '2021.08', '2021.09', '2021.1', '2021.11',
       '2021.12', '2022.01', '2022.02', '2022.03', '2022.04', '2022.05',
       '2022.06'],
      dtype='object')

In [114]:
data5 = pd.merge(data4, pop, how = 'outer', left_on='전입지별', right_on='행정구역(시군구)별')
# data5 = data5.drop(['행정구역(시군구)별'], axis=1)
data5.head()

,전출지별,전입지별,2017.01_x,2017.02_x,2017.03_x,2017.04_x,2017.05_x,2017.06_x,2017.07_x,2017.08_x,...,2021.09_y,2021.1_y,2021.11_y,2021.12_y,2022.01_y,2022.02_y,2022.03_y,2022.04_y,2022.05_y,2022.06_y
0,부산광역시,서울특별시,1800,2871,2236,1413,1380,1398,1330,1779,...,9542256,9532428,9520880,9509458,9505926,9508451,9506778,9500480,9496887,9494807
1,대구광역시,서울특별시,1213,2008,1550,977,1009,856,844,1166,...,9542256,9532428,9520880,9509458,9505926,9508451,9506778,9500480,9496887,9494807
2,인천광역시,서울특별시,2704,3761,3207,2540,2858,2567,2506,2810,...,9542256,9532428,9520880,9509458,9505926,9508451,9506778,9500480,9496887,9494807
3,광주광역시,서울특별시,1008,1562,1261,786,761,757,709,950,...,9542256,9532428,9520880,9509458,9505926,9508451,9506778,9500480,9496887,9494807
4,대전광역시,서울특별시,1431,2195,1590,948,924,995,928,1206,...,9542256,9532428,9520880,9509458,9505926,9508451,9506778,9500480,9496887,9494807


In [115]:
data6 = pd.merge(data5, pop, how = 'outer', left_on='전출지별', right_on='행정구역(시군구)별')
# data5 = data5.drop(['행정구역(시군구)별'], axis=1)
data6.head()

,전출지별,전입지별,2017.01_x,2017.02_x,2017.03_x,2017.04_x,2017.05_x,2017.06_x,2017.07_x,2017.08_x,...,2021.09,2021.1,2021.11,2021.12,2022.01,2022.02,2022.03,2022.04,2022.05,2022.06
0,부산광역시,서울특별시,1800,2871,2236,1413,1380,1398,1330,1779,...,3358763,3356311,3353627,3350380,3348874,3347396,3343504,3340214,3338167,3336737
1,부산광역시,대구광역시,425,642,477,319,342,303,333,320,...,3358763,3356311,3353627,3350380,3348874,3347396,3343504,3340214,3338167,3336737
2,부산광역시,인천광역시,263,288,334,243,261,233,215,243,...,3358763,3356311,3353627,3350380,3348874,3347396,3343504,3340214,3338167,3336737
3,부산광역시,광주광역시,84,138,134,84,63,73,93,110,...,3358763,3356311,3353627,3350380,3348874,3347396,3343504,3340214,3338167,3336737
4,부산광역시,대전광역시,222,300,255,157,157,162,147,153,...,3358763,3356311,3353627,3350380,3348874,3347396,3343504,3340214,3338167,3336737


In [116]:
data6.to_csv('tem3.csv', encoding='utf-8-sig')